In [1]:
import actonet
import biolqm

This notebook has been executed using the docker image `algorecell/algorecell-docker` built on `2019-12-06`

In [2]:
lqm = biolqm.load("http://ginsim.org/sites/default/files/Bladder_Model.zginml")
bn = biolqm.to_minibn(biolqm.booleanize(lqm)).simplif(y)
bn

Downloading http://ginsim.org/sites/default/files/Bladder_Model.zginml

AKT <- PI3K
ATM_b1 <- (!DNAdamage&ATM_b1&ATM_b2)|DNAdamage
ATM_b2 <- DNAdamage&E2F1_b1&ATM_b1
Apoptosis_b1 <- (!Apoptosis_b1&!E2F1_b1&TP53)|(!Apoptosis_b1&E2F1_b1&!E2F1_b2&TP53)|(!Apoptosis_b1&E2F1_b1&E2F1_b2)|(Apoptosis_b1&!Apoptosis_b2&!E2F1_b1&TP53)|(Apoptosis_b1&!Apoptosis_b2&E2F1_b1&!E2F1_b2&TP53)|(Apoptosis_b1&!Apoptosis_b2&E2F1_b1&E2F1_b2)|(Apoptosis_b1&Apoptosis_b2)
Apoptosis_b2 <- Apoptosis_b1&E2F1_b1&E2F1_b2
CDC25A <- (!E2F1_b1&E2F3_b1&!RBL2&!CHEK1_2_b1)|(E2F1_b1&!RBL2&!CHEK1_2_b1)
CHEK1_2_b1 <- (!ATM_b1&CHEK1_2_b1&CHEK1_2_b2)|ATM_b1
CHEK1_2_b2 <- E2F1_b1&ATM_b1&CHEK1_2_b1
CyclinA <- (!E2F1_b1&E2F3_b1&CDC25A&!RBL2&!p21CIP)|(E2F1_b1&CDC25A&!RBL2&!p21CIP)
CyclinD1 <- (!RAS&!p16INK4a&!p21CIP&AKT)|(RAS&!p16INK4a&!p21CIP)
CyclinE1 <- (!E2F1_b1&E2F3_b1&CDC25A&!RBL2&!p21CIP)|(E2F1_b1&CDC25A&!RBL2&!p21CIP)
DNAdamage <- DNAdamage
E2F1_b1 <- (!RAS&!E2F1_b1&E2F3_b1&!RB1&!RBL2)|(!RAS&E2F1_b1&!E2F1_b2&E2F3_b1&!RB1&!RBL2)|(!RAS&E2F1_b1&E2F1_b2)|(RAS&!E2F1_b1&!RB1&!RBL2)|(RAS&E2F1_b1&!E2F1_

In [ ]:
act = actonet.load(bn)

In [ ]:
inputs = {"DNAdamage": 0, "FGFR3_stimulus": 1, "EGFR_stimulus": 0, "GrowthInhibitors": 1}
spec = {"Proliferation": 1, "Apoptosis_b1": 0, "Apoptosis_b2": 0, "Growth_Arrest": 0}

In [3]:
act.set_outputs(list(spec.keys()))
act.set_input_condition(inputs)
act.set_property(spec)
for sol in act.controls(maxcontrol=3):
    print(sol)

{'DNAdamage': 0, 'p16INK4a': 0, 'GRB2': 1}
{'DNAdamage': 0, 'RBL2': 0, 'GRB2': 1}
{'ATM_b1': 0, 'p16INK4a': 0, 'GRB2': 1}
{'ATM_b1': 0, 'RBL2': 0, 'GRB2': 1}
{'ATM_b1': 0, 'CyclinD1': 1, 'GRB2': 1}
{'ATM_b1': 0, 'CyclinE1': 1}
{'ATM_b2': 0, 'TP53': 0, 'CyclinE1': 1}
{'CHEK1_2_b2': 0, 'TP53': 0, 'CyclinE1': 1}
{'FGFR3_stimulus': 0, 'TP53': 0, 'CyclinE1': 1}
{'E2F1_b1': 0, 'TP53': 0, 'CyclinE1': 1}
{'RAS': 0, 'TP53': 0, 'CyclinE1': 1}
{'E2F1_b2': 0, 'TP53': 0, 'CyclinE1': 1}
{'CHEK1_2_b1': 0, 'p16INK4a': 0, 'GRB2': 1}
{'CHEK1_2_b1': 0, 'CyclinE1': 1}
{'DNAdamage': 0, 'CyclinE1': 1}
{'DNAdamage': 0, 'CyclinD1': 1, 'GRB2': 1}
{'FGFR3': 0, 'TP53': 0, 'CyclinE1': 1}
{'CHEK1_2_b1': 0, 'CyclinD1': 1, 'GRB2': 1}
{'CHEK1_2_b1': 0, 'RBL2': 0, 'GRB2': 1}
{'CHEK1_2_b1': 0, 'GrowthInhibitors': 0, 'EGFR_stimulus': 1}
{'DNAdamage': 0, 'GrowthInhibitors': 0, 'EGFR_stimulus': 1}
{'DNAdamage': 0, 'GrowthInhibitors': 0, 'SPRY': 1}
{'CHEK1_2_b1': 0, 'GrowthInhibitors': 0, 'SPRY': 1}
{'DNAdamage': 0, 'Growt

In [4]:
act.asp_fixpoint()

['vAKT :- holds(set(vAKT, 1)).',
 'vAKT :- not holds(set(vAKT, _)), not not vPI3K.',
 'vATM_b1 :- holds(set(vATM_b1, 1)).',
 'vATM_b1 :- not holds(set(vATM_b1, _)), not not vDNAdamage.',
 'vATM_b1 :- not holds(set(vATM_b1, _)), not not vATM_b1, not not vATM_b2.',
 'vATM_b2 :- holds(set(vATM_b2, 1)).',
 'vATM_b2 :- not holds(set(vATM_b2, _)), not not vATM_b1, not not vDNAdamage, not not vE2F1_b1.',
 'vApoptosis_b1 :- not not vApoptosis_b1, not not vApoptosis_b2.',
 'vApoptosis_b1 :- not vApoptosis_b1, not not vE2F1_b1, not not vE2F1_b2.',
 'vApoptosis_b1 :- not vApoptosis_b1, not vE2F1_b1, not not vTP53.',
 'vApoptosis_b1 :- not vApoptosis_b1, not vE2F1_b2, not not vTP53.',
 'vApoptosis_b1 :- not vApoptosis_b2, not not vE2F1_b1, not not vE2F1_b2.',
 'vApoptosis_b1 :- not vApoptosis_b2, not vE2F1_b1, not not vTP53.',
 'vApoptosis_b1 :- not vApoptosis_b2, not vE2F1_b2, not not vTP53.',
 'vApoptosis_b2 :- not not vApoptosis_b1, not not vE2F1_b1, not not vE2F1_b2.',
 'vCDC25A :- holds(set(v

In [5]:
act.asp_guess()

['value(0;1).',
 'variable(vAKT; vATM_b1; vATM_b2; vApoptosis_b1; vApoptosis_b2; vCDC25A; vCHEK1_2_b1; vCHEK1_2_b2; vCyclinA; vCyclinD1; vCyclinE1; vDNAdamage; vE2F1_b1; vE2F1_b2; vE2F3_b1; vE2F3_b2; vEGFR; vEGFR_stimulus; vFGFR3; vFGFR3_stimulus; vGRB2; vGrowthInhibitors; vGrowth_Arrest; vMDM2; vPI3K; vPTEN; vProliferation; vRAS; vRB1; vRBL2; vSPRY; vTP53; vp14ARF; vp16INK4a; vp21CIP).',
 'output(vApoptosis_b1; vApoptosis_b2; vGrowth_Arrest; vProliferation).',
 'controlled(X) :- variable(X), not output(X).',
 'control(X,T) :- controlled(X), value(T).',
 '#const maxcontrol=2.',
 '{set(X,V) : control(X,V)} maxcontrol.',
 ':- set(X,0), set(X,1).',
 'holds(set(X,V)) :- set(X,V).',
 '#show holds/1.',
 '#show nholds/1.',
 '#project holds/1.',
 '{}.']

In [6]:
act.asp_check()

[':- not vApoptosis_b1, not vApoptosis_b2, not vGrowth_Arrest, vProliferation.']